This notebook implements a leaky integrate-and-fire neuron in discrete time suitable for usage in a spiking neural network

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
def heaviside(x, thresh=1.):
    out = np.zeros_like(x)
    out[(x-thresh) > 0.] = 1.
    return out

In [ ]:
n_inputs = 100
n_steps = 5000
dt = 0.5e-3
tend = n_steps * dt
print(f'Simulation time: {tend*1e3:g} ms.')

In [ ]:
freq = 5 # [Hz]
prob = freq * dt
inputs = np.zeros((n_steps, n_inputs))
inputs[np.random.uniform(size=inputs.shape) < prob] = 1
print(f'Total number of input spikes: {inputs.sum()}.')

In [ ]:
tau_mem = 10e-3
tau_syn = 5e-3
alpha   = np.exp(-dt/tau_syn)
beta    = np.exp(-dt/tau_mem)

In [ ]:
weight_scale = 7 * (1.0-beta) # this should give us some spikes to begin with
weights = np.random.normal(loc=0., scale=weight_scale/np.sqrt(n_inputs), size=(n_inputs,))

In [ ]:
Iinp = inputs @ weights
Isyn = np.zeros(n_steps)
Vm = np.zeros(n_steps)
θ = 1.
n_spikes = 0
for t in range(n_steps-1):
    reset = heaviside(Vm[t], θ)
    Isyn[t+1] = alpha * Isyn[t] + Iinp[t]
    # the following line is the one that originally appears in the first tutorial...
    #Vm[t+1] = (beta * Vm[t] + Isyn[t]) * (θ - reset)
    # while this one is what one would get from converting the LIF diff. eq. into a discrete equation
    Vm[t+1] = beta * Vm[t] + Isyn[t] - reset
    if reset > 0:
        n_spikes += 1
        sys.stdout.write('.')
        if n_spikes % 100 == 0:
            sys.stdout.write('\n')
Vm[Vm > θ] = 5*θ

In [ ]:
t = np.arange(n_steps) * dt
fig,ax = plt.subplots(3, 1, figsize=(5,5), sharex=True)
ax[0].plot(t, Vm, 'k', lw=1)
ax[1].plot(t, Isyn, 'k', lw=1)
ax[2].plot(t, Iinp, 'k', lw=1)
for a in ax:
    a.grid(which='major', axis='y', ls=':', lw=0.75, color=[.6,.6,.6])
ax[0].set_ylabel('Vm')
ax[1].set_ylabel('I_syn')
ax[2].set_ylabel('I_inp')
ax[-1].set_xlabel('Time (s)')
sns.despine()
fig.tight_layout()